In [3]:
import tensorflow as tf
from tqdm import tqdm
import os
import re
import glob

In [37]:
class FolderOrganizer:
    def __init__(self, main_folder, keywords):
        self.main_folder = main_folder
        self.keywords = keywords
    
    def create_folders(self):
        for keyword in self.keywords:
            path = os.path.join(self.main_folder, keyword)
            if not os.path.exists(path):
                os.makedirs(path)
                print(f"Folder '{keyword}' telah dibuat.")
            else:
                print(f"Folder '{keyword}' sudah ada.")
    
    def move_files(self):
        files = [f for f in os.listdir(self.main_folder) if os.path.isfile(os.path.join(self.main_folder, f))]
        if len(files) != 0:
            file_count = sum([len(files) for _, _, files in os.walk(self.main_folder)])
            progress_bar = tqdm(total=file_count, desc='Moving files', unit='file')
            for filename in os.listdir(self.main_folder):
                file_path = os.path.join(self.main_folder, filename)
                if os.path.isfile(file_path):
                    for keyword in self.keywords:
                        if re.search(keyword, filename, re.IGNORECASE):
                            self._move_file(filename, keyword)
                            progress_bar.update(1)
                            break
        
    def _move_file(self, filename, folder):
        src_path = os.path.join(self.main_folder, filename)
        dest_path = os.path.join(self.main_folder, folder, filename)
        os.rename(src_path, dest_path)

if __name__ == "__main__":
    keywords = ['blur', 'orig', 'psf']
    organizer = FolderOrganizer('data', keywords)
    organizer.create_folders()
    organizer.move_files()
    print("File telah dipindahkan sesuai dengan folder yang benar.")

Folder 'blur' sudah ada.
Folder 'orig' sudah ada.
Folder 'psf' sudah ada.
File telah dipindahkan sesuai dengan folder yang benar.


In [50]:
class ImageLoader:
    def __init__(self, base_path, extensions, target_size):
        self.base_path = base_path
        self.extensions = extensions
        self.target_size = target_size
        self.images = []

    def load_images_through_walk_folders(self):
        for extension in self.extensions:
            desc = f"Loading {extension.upper()} files"
            for filepath in tqdm(sorted(glob.glob(f"{self.base_path}**/*{extension.lower()}", recursive=True)), desc=desc, unit=' file'):
                image = tf.keras.preprocessing.image.load_img(filepath)
                image = tf.image.resize(
                    image,
                    size=self.target_size,
                    antialias=True,
                    method=tf.image.ResizeMethod.BILINEAR
                )
                image = tf.cast(image, tf.uint8)
                self.images.append(image)
        
        self.images = tf.convert_to_tensor(self.images)
        return self.images

if __name__ == "__main__":
    PARENT_PATH = "./blur_psf_sharp_images/"
    target_size = (200, 200)
    image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff', '.tif', '.webp', '.svg', '.exif']
    
    image_loader = ImageLoader(PARENT_PATH, image_extensions, target_size)
    my_images = image_loader.load_images_through_walk_folders()

Loading .JPG files: 0 file [00:00, ? file/s]
Loading .JPEG files: 0 file [00:00, ? file/s]
Loading .PNG files: 100%|██████████| 528/528 [00:02<00:00, 226.66 file/s]
Loading .GIF files: 0 file [00:00, ? file/s]
Loading .BMP files: 0 file [00:00, ? file/s]
Loading .TIFF files: 0 file [00:00, ? file/s]
Loading .TIF files: 0 file [00:00, ? file/s]
Loading .WEBP files: 0 file [00:00, ? file/s]
Loading .SVG files: 0 file [00:00, ? file/s]
Loading .EXIF files: 0 file [00:00, ? file/s]
